# Zadanie 9

Zaimplementować:
* funkcję `binary_crossentropy_pseudo_residuals`,
* klasę `GammaLoss`,
* klasę `GradientBoostingClassifier`.

Dla uproszczenia zakładamy, że etykiety (y) są binarne, tzn. rozwiązujemy problem dwuklasowy. Predykcja modelu dotyczy klasy oznaczonej numerem 1 (podobnie jak w wypadku regresji logistycznej). Funkcją kosztu, którą chcemy optymalizować, jest więc binarna cross-entropia.

http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html

https://en.wikipedia.org/wiki/Gradient_boosting#Algorithm

https://en.wikipedia.org/wiki/Gradient_boosting#Gradient_tree_boosting

Zakładamy, że każdy składowy model naszego klasyfikatora przewiduje __logits__ (oznaczane $\hat{l}$), a nie prawdopodobieństwa! Dlatego też nasza funkcja kosztu będzie miała następującą postać:

$$L(y,\hat{l}) = -(y\ln\sigma(\hat{l}) + (1-y)\ln(1-\sigma(\hat{l})))$$

Takie założenie jest o tyle wygodne, że skala logitsów jest nieograniczona (w przeciwieństwie do skali prawdopodobieństw), więc możemy "bezkarnie" dodawać predykcje wielu modeli.

Uwaga (przypomnienie) - będziemy budowali klasyfikator z modeli __regresyjnych__.

### `GradientBoostingClassifier`

Klasa `GradientBoostingClassifier` przyjmuje w `__init__`:
* `X` - tablicę o wymiarach (liczba_obserwacji, liczba_cech),
* `y` - tablicę o wymiarach (liczba_obserwacji) z numerami klas (0 lub 1),
* `n_models` - liczbę modeli, które mają zostać wytrenowane,
* `model_cls` - klasę pojedynczego modelu regresyjnego, która przyjmuje w `__init__` jedynie `X` i `y` oraz ma zaimplementowaną metodę `predict` (patrz notebook 11a),
* `train_fraction` - oznacza, jaki ułamek obserwacji ma być wylosowany do uczenia każdego ze składowych modeli (zredukowana_liczba_obserwacji = liczba_obserwacji * train_fraction), losujemy __bez powtórzeń__,
* `initial_gamma` - wartość parametru $\gamma$, od której rozpoczynamy optymalizację `GammaLoss`,
* `gamma_n_steps` - liczba kroków optymalizacji `GammaLoss`,
* `seed` - potrzebny np. przy losowaniu podzbioru obserwacji.

Parametr `models` przechowuje listę nauczonych modeli w kolejności uczenia.

Parametr `gammas` przechowuje listę znalezionych wartości $\gamma$ w kolejności uczenia.

Uczenie przebiega w następującej pętli (wykonywanej `n_models` razy):
1. Wylosuj podzbiór obserwacji.
2. Oblicz dotychczasową predykcję na tym podzbiorze.
3. Oblicz pseudo residua `r_true` (patrz sekcja `binary_crossentropy_pseudo_residuals`).
4. Wytrenuj kolejny model tak, aby przewidywał `r_true`, oblicz jego faktyczne predykcje `r`.
5. Znajdź parametr $\gamma$ (patrz sekcja `GammaLoss`).
6. Dopisz model oraz parametr $\gamma$ do `self.models` i `self.gammas`.

Metoda `predict_logits` ma za zadanie przewidzieć wartości `logits` na danych `X`. Parametr `step` oznacza, ile (pierwszych w kolejności uczenia) modeli składowych użyć do predykcji. Będzie to przydatne przy uczeniu oraz wizualizowaniu nauczonego modelu. Domyślna wartość `None` oznacza użycie wszystkich modeli.

### `binary_crossentropy_pseudo_residuals`

Funkcja `binary_crossentropy_pseudo_residuals` przyjmuje dwie jednowymiarowe tablice `y_true` oraz `logits` o wymiarach (liczba_obserwacji,) i zwraca tablicę takiego samego wymiaru, która zawiera policzone element-wise pochodne cząstkowe $\dfrac{\partial L}{\partial\hat{l}}$ [wyprowadzamy wzór na pochodną przy tablicy].

### `GammaLoss`

Załóżmy, że wytrenowaliśmy już $k$ modeli składowych i chcemy dodać model $m_{k+1}$. W tym celu uczymy model $m_{k+1}$ na odpowiednio dobranych danych. Pozostaje jeszcze dobór parametru $\gamma$. W tym celu rozwiązujemy jednowymiarowy problem optymalizacji przy użyciu klasy `GammaLoss` i dowolnego optimizera.

Klasa `GammaLoss` przyjmuje w `__init__`:
* `y_true` - tablica o wymiarze (zredukowana_liczba_obserwacji,) zawierająca prawdziwe etykiety,
* `logits` - tablica o wymiarze (zredukowana_liczba_obserwacji,), dotychczasowa łączna predykcja modeli,
* `r` - tablica o wymiarze (zredukowana_liczba_obserwacji,), predykcja modelu $m_{k+1}$
i zapamiętuje te tablice - będą one potrzebne w metodzie `taylor`.

Metoda `taylor` przyjmuje wartość $\gamma$ i zwraca:
* wartość kosztu $L(\gamma) = \mathcal{średnia}(-(y\ln\sigma(l+\gamma r) + (1-y)\ln(1-\sigma(l+\gamma r))))$
* pochodną $\dfrac{\partial L}{\partial\gamma}$ [wyprowadzamy wzór na pochodną przy tablicy]